In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from pathlib import Path
import re 

import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blpw import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import apply_function
from api.bbg import BBGRiskFreeCurve, BBGCDS
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
    get_curve_members
)

In [3]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

#### Local data

In [4]:
curve_hist = pd.read_csv(data_path / 'curve_history.csv', parse_dates=['date'], dayfirst=True)
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)
cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

#### Settings

In [5]:
cds_name = 'TURKEY CDS USD SR 5Y D14 Curncy'
index_name = 'CDX EM CDSI S37 5Y PRC Corp'
curve_name = 'USD ISDA CDS SOFR Rates Swap Curve' #  "USD ISDA Standard Rates Swap Curve" # "EUR ISDA Standard Rates Swap Curve"
trade_date = pd.Timestamp.today().floor('d')
pcs = 'MSG1'
sep = ' '

date_start = pd.Timestamp('2021-06-02')
date_end = pd.Timestamp('2022-06-23')

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

#### Bloomberg Connection

In [6]:
bq = BlpQuery(timeout=50000).start()

In [7]:
cds_name = 'CT350188 MSG1 Curncy'
curve_name = 'USD ISDA CDS SOFR Rates Swap Curve'
start_date = '20220530'

In [8]:
# members = pd.read_clipboard(sep=',', header=None).squeeze()
cds = BBGCDS(cds_name, bq=bq)
members = get_curve_members(curve_name, curves)
rfc = BBGRiskFreeCurve(members=members, bq=bq)

#### Current Data

In [9]:
trade_date = pd.Timestamp.today().floor('d')
notional = 10000000

In [10]:
rfc_mkt = rfc.get_mkt()
cds_ref = cds.get_ref()
cds_mkt = cds.get_mkt()

In [13]:
cds_mkt

security          CT350188 MSG1 Curncy
px_last                        7907.92
upfront_last                      76.5
trades_upfront                    True
Name: 0, dtype: object

#### Pricing

In [14]:
spread = cds_mkt['px_last'] if not cds_mkt.get('trades_upfront', False) else None
upfront = cds_mkt['upfront_last'] if cds_mkt.get('trades_upfront', False) else 0
cds = get_isda_cds(
    risk_free_curve=rfc_mkt,
    trade_date=trade_date,
    cds_spread=spread,
    tenor=cds_ref['tenor'],
    recovery=cds_ref['recovery'],
    upfront=upfront,
    notional=notional,
    coupon=cds_ref['coupon'],
    rebates_accrual=True
)

In [ ]:
cds